In [24]:
import json
from utils.chat import *


# load 1st line of data/dev.json
with open('data/dev.json') as f:
    data = json.load(f)
    i = 0
    datapoint = data[i]
    print(datapoint['claim'])
    print(datapoint['label'])
    print(datapoint['justification'])
    
system_prompt = """You are a professional fact checker, formulate about 4 questions that cover all the facts needed to validate whether the following statement is true, false, uncertain or a matter of opinion.

Please, you MUST only print the output in the following output format:
```json
[
    "<Your first question>",
    "<Your second question>",
    "<Your third question>"
]
```"""
user_message = datapoint['claim']
chat = SimpleJSONChat(model="gpt-4o",system_prompt=system_prompt)

In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.
Refuted
The answer and sources show that the claim was published in a fake news site so the claim is refuted.


In [26]:
print(chat(user_message))

['Did Sean Connery ever write a letter to Steve Jobs?', 'Did Sean Connery refuse to appear in an Apple commercial?', 'Is there any documented evidence or credible source that confirms the existence of such a letter?']


In [27]:
chat("The United States of America and its Western allies have been using their media outlets to publish articles based on fabricated information under allegations of non-compliance with the Chemical Weapons Convention.")

['Have there been verified instances of the United States or its Western allies using media outlets to publish fabricated information?',
 'What specific articles or media reports are being referred to in the statement?',
 'Have there been any official investigations or reports confirming the use of fabricated information by the United States or its Western allies regarding the Chemical Weapons Convention?',
 'What are the sources or evidence supporting the claim that the information published was fabricated?']

In [28]:
chat( "Why should you pay more taxes than Donald Trump pays? And that\u2019s a fact. $750. Remember what he said when that was raised a while ago, how he only pays \u2026 He said, \u2018Because I\u2019m smart. I know how to game the system.\u2019")

['Did Donald Trump pay $750 in federal income taxes in a specific year?',
 'Did Donald Trump publicly state that he pays low taxes because he is smart and knows how to game the system?',
 'Is there a context or specific time frame mentioned when Donald Trump made the statement about paying low taxes?']